# Mortgage Risk Analyzer v4

## Intro
I created this document to help myself and others realized the financtal implications of buying a home.

Hypothysis: You spend 1.5 times the amount advertized on a home.

In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
.jp-OutputArea {
    overflow-y: unset !important;
}
.widget-box {
    overflow: unset !important;
}
</style>

In [2]:
import ipywidgets as widgets
input_sliders = []
input_sliders.append(widgets.FloatSlider(value=3,min=1,max=10,step=0.01,description='Intrest Rate (%/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.IntSlider(value=300,min=100,max=1000,description='House Price ($k)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.IntSlider(value=60,min=1,max=100,description='Down Payment ($k)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.IntSlider(value=30,min=1,max=60,description='Term (years)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.FloatSlider(value=2,min=0,max=5,description='Private Mortgage Insurance (%/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.FloatSlider(value=1,min=1,max=5,description='Payment Multiplier (%)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
input_sliders.append(widgets.IntSlider(value=1500,min=500,max=5000,description='Specific Payment Amount ($)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))

onetime_sliders = []
onetime_sliders.append(widgets.FloatSlider(value=2,min=1,max=5,description='Closing Costs (%)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
onetime_sliders.append(widgets.IntSlider(value=350,min=300,max=400,description='Appraisal Fee ($)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
onetime_sliders.append(widgets.IntSlider(value=400,min=300,max=500,description='Inspection Fee ($)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))

lifetime_sliders = []
lifetime_sliders.append(widgets.IntSlider(value=600,min=500,max=5000,description='Home Insurance ($/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
lifetime_sliders.append(widgets.IntSlider(value=200,min=0,max=700,description='HOA contribution ($/mo)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
lifetime_sliders.append(widgets.FloatSlider(value=4.5,min=0,max=200,description='Property Taxes ($k/yr)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))
lifetime_sliders.append(widgets.IntSlider(value=300,min=0,max=1000,description='Utilities ($/mo)',style={'description_width': 'initial'},layout=widgets.Layout(width='100%')))


In [3]:
def calculateX(intrestRate, housePrice, downPayment, term, privateMortgageInsurancePct=0, minPaymentMuliplyer=1, specificPaymentAmount=None):
    intrestRate = intrestRate/100
    housePrice = housePrice*1000
    downPayment = downPayment*1000
    privateMortgageInsurancePct = privateMortgageInsurancePct/100/12 # monthly pmi
    
    #print('Calculating x for ',intrestRate,' ',housePrice,' ',downPayment,' ',term,' ',minPaymentMuliplyer)
    n = 12*term
    monthlyInterestRate = intrestRate / 12
    #print("monthlyInterestRate =",monthlyInterestRate)
    r = 1/(1+monthlyInterestRate)
    loan = housePrice - downPayment
    #print("loan =",loan)
    minPayment = loan * ((1-r)/(r-r**(n+1)))
    #print("minpayment =",minPayment)
    payment = minPayment * minPaymentMuliplyer
    if specificPaymentAmount:
        payment = specificPaymentAmount
    #print("payment =",payment)
    
    paymentSum = 0
    totalmonths = 0
    totalmonthsWithPMI = 0
    totalprivateMortgageInsurance = 0
    balance = loan
    
    while balance > 1:
        #print('balance ',balance)
        # 1 month passes
        interest = balance * monthlyInterestRate
        #print('interest ',interest)
        
        p = payment
        #print('payment ',p)
        if interest + balance < payment:
            p = interest + balance
        
        privateMortgageInsurance = 0
        if ((paymentSum + downPayment) < loan * 0.2) and (totalmonths < term*12/2):
            privateMortgageInsurance = privateMortgageInsurancePct*loan
            totalmonthsWithPMI += 1
            totalprivateMortgageInsurance += privateMortgageInsurance
        
        if p < interest:
            #print('error. payment too small ',intrestRate,' ',housePrice,' ',downPayment,' ',term,' ',minPaymentMuliplyer)
            return 0,0,0,0,0,0
            
        balance = balance - (p) + (interest)
        paymentSum += p
        totalmonths += 1
    expenses = paymentSum + downPayment + totalprivateMortgageInsurance
    x = expenses / housePrice
    return x, totalmonths, minPayment, payment, totalmonthsWithPMI, totalprivateMortgageInsurance
    
        

In [4]:
def calculateOneTimeExpenses(housePrice, downPayment, closingCostsPrct=0, appraisalFee=0, inspectionFee=0):
    housePrice = housePrice*1000
    downPayment = downPayment*1000
    closingCostsPrct = closingCostsPrct/100
    loan = housePrice - downPayment
    closingCosts = closingCostsPrct*loan
    return closingCosts + appraisalFee + inspectionFee

In [5]:
def calculateLifetimeExpensesPerMonth(homeInsurance,HOA,propertyTaxes,utilities):
    return homeInsurance/12 + HOA + propertyTaxes*1000/12 + utilities


In [6]:
def input_slider_values():
    values = []
    for s in input_sliders:
        values.append(s.value)
    return values
def onetime_slider_values():
    values = []
    for s in onetime_sliders:
        values.append(s.value) 
    return values
def lifetime_slider_values():
    values = []
    for s in lifetime_sliders:
        values.append(s.value)
    return values


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
def mccgraph(x,y,xlabel):
    #plt.figure()
    plt.xlabel(xlabel)
    plt.ylabel('x')
    plt.plot(x, y, '-ok')
    plt.show()

In [8]:
def show_comparitive_graphs(v0,v1,v2,v3,v4,v5,v6):
    variables = []
    for s in input_sliders:
        smin = int(s.min*100)
        smax = int(s.max*100)
        diff = smax-smin
        points_per_graph = 20
        step = int(diff/points_per_graph)
        variables.append([x / 100 for x in range(smin, smax, step)])
    for i,var in enumerate(variables):
        #print(var)
        values = input_slider_values()
        output = []
        xlabel = ''
        xvalues = []
        for v in var:
            #print(v)
            values[i] = v
            xlabel = input_sliders[i].description
            x, totalmonths, minPayment, payment, totalmonthsWithPMI, totalPMI = calculateX(values[0],values[1],values[2],values[3],values[4],values[5],values[6])
            if x>0:
                output.append(x)
                xvalues.append(v)
        mccgraph(xvalues, output, xlabel)
    #plt.show()

In [9]:
def show_output(v0,v1,v2,v3,v4,v5,v6):
    show_comparitive_graphs(v0,v1,v2,v3,v4,v5,v6)

interactive_plot = widgets.interactive(show_output
, v0=input_sliders[0]
, v1=input_sliders[1]
, v2=input_sliders[2]
, v3=input_sliders[3]
, v4=input_sliders[4]
, v5=input_sliders[5]
, v6=input_sliders[6])
for index, child in enumerate(interactive_plot.children):
    if index > 6:
        child.layout.height = '350px'

display(interactive_plot)

interactive(children=(FloatSlider(value=3.0, description='Intrest Rate (%/yr)', layout=Layout(width='100%'), m…